In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    05/11/2025  -  02:55:40           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Competición Perretes y Gatos

## Iteración 6 - Ensemble Learning + TTA + Imagenes mas grandes

Mejoró el resultado del anterior pero los 2 modelos inferiores a B3 en tamaño arrastraron la precisión final impidiendo una mejora drastica.

El siguiente paso y visto el exito de ImageNetB3 será tirar por modelos de esa familia renunciando a mi pesar Esemble learning. 

**kaggle score**: 0.90858, aunque mejor se esperaba una mejora mucho mas considerable y gran parte de la precisión es gracias a B3


In [ ]:
# Imports para ensemble learning
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.applications import VGG16, EfficientNetB3, ResNet50

seed = 42
keras.utils.set_random_seed(seed)
np.random.seed(seed)

# Rutas dataset
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("="*70)
print("ITERACION 6 - ENSEMBLE LEARNING")
print("="*70)
print("Keras:", keras.__version__)
print("="*70)

## Config

Pra pruebas:

In [ ]:
USE_HIGH_RES = False  # True = 384x384 (mejor pero 3x más lento), False = 224x224
N_TTA_AUGMENTATIONS = 5  # Augmentations por imagen en TTA (reducido para que velocidad)
ENSEMBLE_MODELS = ['vgg16', 'efficientnet', 'resnet']  # Modelos a usar en ensemble

# Parámetros q cambian según resolución
IMG_SIZE = 384 if USE_HIGH_RES else 224
BATCH_SIZE = 32 if USE_HIGH_RES else 125

print(f"Resolución: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"TTA augmentations: {N_TTA_AUGMENTATIONS}")
print(f"Modelos ensemble: {ENSEMBLE_MODELS}")

## Carga de Datos

Cargo las imágenes y las divido:
- **80% train**: entrenar
- **20% validation**:  validar q no se me sobreajuste

Todo se redimensiona según la config (224x224 o 384x384).

In [ ]:
# Cargo datos con la resolución configurada
train_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
)

validation_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
)

test_dataset = keras.utils.image_dataset_from_directory(
    TEST_PATH,
    labels=None,
    label_mode=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

supplementary_dataset = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

print("="*70)
print(f"Train: {len(train_dataset)} batches")
print(f"Validation: {len(validation_dataset)} batches")
print(f"Test: {len(test_dataset)} batches")
print(f"Supplementary: {len(supplementary_dataset)} batches")
print("="*70)

## Data Augmentation

Transformaciones random pa q el modelo no memorice:
- **RandomFlip**: voltea horizontal
- **RandomRotation**: rota ±10%
- **RandomZoom**: zoom ±10%
- **RandomTranslation**: mueve ±10%

Conservador porque usamos Transfer Learning, no hace falta pasarse.

In [ ]:
# Augmentation conservador para Transfer Learning
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
], name="data_augmentation")

print("Data Augmentation listo")

In [ ]:
# Aplico augmentation a los datasets
train_dataset_augmented = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
validation_dataset_augmented = validation_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

print("Datasets con augmentation listos")

## MODELO 1/3: VGG16

Transfer Learning clásico:
1. Cargo VGG16 pre-entrenado en ImageNet (14M imágenes)
2. Congelo todas las capas
3. Añado mi cabecera:
   - Dense(256) + Dropout(0.5) + Dense(1, sigmoid)

Solo entreno ~67K parámetros en vez de 15M. Mucho más rápido.

In [ ]:
# Cargo VGG16 pre-entrenado
vgg16_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Congelo todo
for layer in vgg16_base.layers:
    layer.trainable = False

# Construyo modelo
vgg16_model = Sequential([
    vgg16_base,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

vgg16_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("="*70)
print("MODELO 1/3: VGG16")
print("="*70)
vgg16_model.summary()
print("="*70)
print(f"Capas entrenables: {sum([layer.trainable for layer in vgg16_base.layers])}/{len(vgg16_base.layers)}")
print("="*70)

## Transfer Learning - VGG16

Entreno con VGG16 congelado:
- Adam optimizer con LR bajo (0.0001)
- 10 épocas (optimizado pa velocidad)
- ReduceLROnPlateau: baja LR si se estanca

Solo entreno las capas Dense q añadí.

In [ ]:
# Callback para medir tiempo
import time

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, epoch, logs={}):
        epoch_time = time.time() - self.epoch_time_start
        self.times.append(epoch_time)
        print(f"\nÉpoca {epoch+1} completada en {epoch_time:.1f}s ({epoch_time/60:.2f} min)")

time_callback = TimeHistory()

# Entreno VGG16 con Transfer Learning
vgg16_history_tl = vgg16_model.fit(
    train_dataset_augmented,
    epochs=10,  # Reducido para que velocidad
    validation_data=validation_dataset_augmented,
    callbacks=[
        time_callback,
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Transfer Learning VGG16 completado")
print(f"Tiempo promedio por época: {sum(time_callback.times)/len(time_callback.times):.1f}s")
print("="*70)

In [ ]:
# Visualizo resultados Transfer Learning VGG16
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(vgg16_history_tl.history['loss'], label='Train')
plt.plot(vgg16_history_tl.history['val_loss'], label='Validation')
plt.title('Loss - VGG16 TL')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(vgg16_history_tl.history['accuracy'], label='Train')
plt.plot(vgg16_history_tl.history['val_accuracy'], label='Validation')
plt.title('Accuracy - VGG16 TL')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evalúo VGG16 en datos suplementarios
vgg16_supp_results = vgg16_model.evaluate(supplementary_dataset, verbose=0)
vgg16_supp_accuracy = vgg16_supp_results[1]  # accuracy es el segundo elemento

print("="*70)
print(f"VGG16 Accuracy en datos suplementarios: {vgg16_supp_accuracy:.4f}")
print("="*70)

In [ ]:
# Descongelo solo el último bloque (block5) de VGG16
vgg16_base.trainable = True

# Congelo todo excepto las últimas 4 capas (block5)
for layer in vgg16_base.layers[:-4]:
    layer.trainable = False

# Verifico
print("Capas VGG16 después de descongelar block5:")
trainable_count = sum([1 for layer in vgg16_base.layers if layer.trainable])
frozen_count = sum([1 for layer in vgg16_base.layers if not layer.trainable])

print(f"Congeladas: {frozen_count}")
print(f"Entrenables: {trainable_count}")
print(f"Total params: {vgg16_model.count_params():,}")

## Fine-tuning VGG16

Ahora descongelo las últimas capas de VGG16 (block5) pa ajustar fino:
- LR MUY bajo (1e-5) pa no romper lo pre-entrenado
- 10 épocas más
- Objetivo: mejorar accuracy

Riesgo: puede empeorar si sobreajusto, por eso LR bajo.

In [ ]:
# Recompilo con LR MUY bajo para fine-tuning
vgg16_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # 10x más bajo
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Entreno con fine-tuning
vgg16_history_ft = vgg16_model.fit(
    train_dataset_augmented,
    epochs=10,  # 10 épocas para todos los modelos
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,  # Menos patience
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Fine-tuning VGG16 completado")
print("="*70)

In [ ]:
# Visualizo resultados Fine-tuning VGG16
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(vgg16_history_ft.history['loss'], label='Train')
plt.plot(vgg16_history_ft.history['val_loss'], label='Validation')
plt.title('Loss - VGG16 FT')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(vgg16_history_ft.history['accuracy'], label='Train')
plt.plot(vgg16_history_ft.history['val_accuracy'], label='Validation')
plt.title('Accuracy - VGG16 FT')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

---

# MODELO 2/3: EfficientNetB3

EfficientNetB3 es un modelo más moderno y eficiente que VGG16, diseñado específicamente para maximizar accuracy con menos parámetros.

**Ventajas**:
- Arquitectura balanceada (depth, width, resolution)
- Mejor relación accuracy/parámetros
- Entrenado en ImageNet (mismo dataset que VGG16)

Vamos a seguir el mismo proceso: **Transfer Learning** → **Fine-tuning**

## Construcción del Modelo

In [ ]:
# Cargo EfficientNetB3 pre-entrenado
efficientnet_base = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Congelo todo
for layer in efficientnet_base.layers:
    layer.trainable = False

# Construyo modelo
efficientnet_model = Sequential([
    efficientnet_base,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

efficientnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("="*70)
print("MODELO 2/3: EfficientNetB3")
print("="*70)
efficientnet_model.summary()
print("="*70)
print(f"Capas entrenables: {sum([layer.trainable for layer in efficientnet_base.layers])}/{len(efficientnet_base.layers)}")
print("="*70)

## Transfer Learning - EfficientNetB3

Entreno con EfficientNetB3 congelado.

In [ ]:
# Entreno EfficientNetB3 con Transfer Learning
efficientnet_history_tl = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=10,  # Reducido para que velocidad
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Transfer Learning EfficientNetB3 completado")
print("="*70)

In [ ]:
# Visualizo resultados Transfer Learning EfficientNetB3
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(efficientnet_history_tl.history['loss'], label='Train')
plt.plot(efficientnet_history_tl.history['val_loss'], label='Validation')
plt.title('Loss - EfficientNetB3 TL')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(efficientnet_history_tl.history['accuracy'], label='Train')
plt.plot(efficientnet_history_tl.history['val_accuracy'], label='Validation')
plt.title('Accuracy - EfficientNetB3 TL')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Fine-tuning - EfficientNetB3

Descongelo últimas 20 capas pa ajustar fino.

In [ ]:
# Descongelo últimas 20 capas
for layer in efficientnet_base.layers[-20:]:
    layer.trainable = True

# Recompilo con LR bajo
efficientnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("="*70)
print(f"Capas entrenables: {sum([layer.trainable for layer in efficientnet_base.layers])}/{len(efficientnet_base.layers)}")
print("="*70)

In [ ]:
# Entreno con fine-tuning EfficientNetB3
efficientnet_history_ft = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=10,  # 10 épocas para todos los modelos
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,  # Menos patience
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Fine-tuning EfficientNetB3 completado")
print("="*70)

In [ ]:
# Visualizo resultados Fine-tuning EfficientNetB3
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(efficientnet_history_ft.history['loss'], label='Train')
plt.plot(efficientnet_history_ft.history['val_loss'], label='Validation')
plt.title('Loss - EfficientNetB3 Fine-tuning')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(efficientnet_history_ft.history['accuracy'], label='Train')
plt.plot(efficientnet_history_ft.history['val_accuracy'], label='Validation')
plt.title('Accuracy - EfficientNetB3 Fine-tuning')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evalúo EfficientNetB3 en datos suplementarios
print("Evaluando EfficientNetB3 en datos suplementarios...")
efficientnet_supp_results = efficientnet_model.evaluate(supplementary_dataset)
efficientnet_supp_accuracy = efficientnet_supp_results[1]

print("="*70)
print(f"EfficientNetB3 Supplementary Accuracy: {efficientnet_supp_accuracy:.4f}")
print("="*70)

---

# MODELO 3/3: ResNet50

ResNet50 introduce skip connections (conexiones residuales) que permiten entrenar redes muy profundas sin degradación.

**Ventajas**:
- Arquitectura profunda (50 capas)
- Skip connections resuelven el problema del gradient vanishing
- Gran capacidad de aprendizaje de features complejas

Mismo proceso: **Transfer Learning** → **Fine-tuning**

## Construcción del Modelo

In [ ]:
# Cargo ResNet50 pre-entrenado
resnet_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Congelo todo
for layer in resnet_base.layers:
    layer.trainable = False

# Construyo modelo
resnet_model = Sequential([
    resnet_base,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

resnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("="*70)
print("MODELO 3/3: ResNet50")
print("="*70)
resnet_model.summary()
print("="*70)
print(f"Capas entrenables: {sum([layer.trainable for layer in resnet_base.layers])}/{len(resnet_base.layers)}")
print("="*70)

## Transfer Learning - ResNet50

Entreno con ResNet50 congelado.

In [ ]:
# Entreno ResNet50 con Transfer Learning
resnet_history_tl = resnet_model.fit(
    train_dataset_augmented,
    epochs=10,  # Reducido para que velocidad
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Transfer Learning ResNet50 completado")
print("="*70)

In [ ]:
# Visualizo resultados Transfer Learning ResNet50
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resnet_history_tl.history['loss'], label='Train')
plt.plot(resnet_history_tl.history['val_loss'], label='Validation')
plt.title('Loss - ResNet50 TL')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(resnet_history_tl.history['accuracy'], label='Train')
plt.plot(resnet_history_tl.history['val_accuracy'], label='Validation')
plt.title('Accuracy - ResNet50 TL')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Fine-tuning - ResNet50

Descongelo últimas 15 capas.

In [ ]:
# Descongelo últimas 15 capas
for layer in resnet_base.layers[-15:]:
    layer.trainable = True

# Recompilo con LR bajo
resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("="*70)
print(f"Capas entrenables: {sum([layer.trainable for layer in resnet_base.layers])}/{len(resnet_base.layers)}")
print("="*70)

In [ ]:
# Entreno con fine-tuning ResNet50
resnet_history_ft = resnet_model.fit(
    train_dataset_augmented,
    epochs=10,  # 10 épocas para todos los modelos
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,  # Menos patience
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("="*70)
print("Fine-tuning ResNet50 completado")
print("="*70)

In [ ]:
# Visualizo resultados Fine-tuning ResNet50
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resnet_history_ft.history['loss'], label='Train')
plt.plot(resnet_history_ft.history['val_loss'], label='Validation')
plt.title('Loss - ResNet50 Fine-tuning')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(resnet_history_ft.history['accuracy'], label='Train')
plt.plot(resnet_history_ft.history['val_accuracy'], label='Validation')
plt.title('Accuracy - ResNet50 Fine-tuning')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evalúo ResNet50 en datos suplementarios
print("Evaluando ResNet50 en datos suplementarios...")
resnet_supp_results = resnet_model.evaluate(supplementary_dataset)
resnet_supp_accuracy = resnet_supp_results[1]

print("="*70)
print(f"ResNet50 Supplementary Accuracy: {resnet_supp_accuracy:.4f}")
print("="*70)

---

# Comparación Modelos

Veo cómo van los 3 en supplementary:

In [ ]:
# Comparo los 3
models_comparison = {
    'VGG16': vgg16_supp_accuracy,
    'EfficientNetB3': efficientnet_supp_accuracy,
    'ResNet50': resnet_supp_accuracy
}

plt.figure(figsize=(10, 6))
plt.bar(models_comparison.keys(), models_comparison.values(), color=['blue', 'green', 'red'])
plt.title('Comparación Supplementary - 3 Modelos')
plt.ylabel('Accuracy')
plt.ylim(0.6, 1.0)
plt.grid(True, axis='y', alpha=0.3)

for i, (model, acc) in enumerate(models_comparison.items()):
    plt.text(i, acc + 0.01, f'{acc:.4f}', ha='center', fontweight='bold')

plt.show()

print("="*70)
print("COMPARACIÓN")
print("="*70)
for model, acc in models_comparison.items():
    print(f"{model:20s}: {acc:.4f}")
print("="*70)

---

# Test Time Augmentation (TTA)

Predigo cada imagen varias veces con augmentations random y promedio.

**Por qué mola**:
- Reduce overfitting
- Más robusto
- +1-2% gratis sin reentrenar

**Cómo va**:
1. Cojo 1 imagen
2. Le hago N augmentations random
3. Predigo cada una → N predicciones
4. Promedio → predicción final

In [ ]:
# Función TTA
def predict_with_tta(model, img_array, n_augmentations=10):
    """Predice con TTA - hace N augmentations y promedia"""
    # Layer de augmentation
    augmentation_layer = keras.Sequential([
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(0.1),
        keras.layers.RandomTranslation(0.1, 0.1),
    ])
    
    predictions = []
    for _ in range(n_augmentations):
        # Augmentation random
        augmented_img = augmentation_layer(img_array, training=True)
        # Predigo
        pred = model.predict(augmented_img, verbose=0)
        predictions.append(pred)
    
    # Promedio
    return np.mean(predictions, axis=0)

print("Función TTA lista")
print(f"Se usarán {N_TTA_AUGMENTATIONS} augmentations por imagen")

---

# Ensemble + TTA - Predicciones

Combino **Ensemble** (3 modelos) + **TTA** (10 augmentations).

**Total por imagen**: 3 × 10 = **30 predicciones**

Promedio todo pa la predicción final.

In [ ]:
# Genero predicciones con Ensemble + TTA
print("="*70)
print("GENERANDO PREDICCIONES")
print("="*70)
print(f"Modelos: {ENSEMBLE_MODELS}")
print(f"TTA por modelo: {N_TTA_AUGMENTATIONS}")
print(f"Total por imagen: {len(ENSEMBLE_MODELS) * N_TTA_AUGMENTATIONS}")
print("="*70)

ensemble_predictions = []

for batch in test_dataset:
    batch_predictions = []
    
    # Predigo con cada modelo + TTA
    if 'vgg16' in ENSEMBLE_MODELS:
        vgg16_pred = predict_with_tta(vgg16_model, batch, N_TTA_AUGMENTATIONS)
        batch_predictions.append(vgg16_pred)
    
    if 'efficientnet' in ENSEMBLE_MODELS:
        efficientnet_pred = predict_with_tta(efficientnet_model, batch, N_TTA_AUGMENTATIONS)
        batch_predictions.append(efficientnet_pred)
    
    if 'resnet' in ENSEMBLE_MODELS:
        resnet_pred = predict_with_tta(resnet_model, batch, N_TTA_AUGMENTATIONS)
        batch_predictions.append(resnet_pred)
    
    # Promedio todo
    ensemble_pred = np.mean(batch_predictions, axis=0)
    ensemble_predictions.extend(ensemble_pred)

ensemble_predictions = np.array(ensemble_predictions)

print("="*70)
print(f"Predicciones listas: {len(ensemble_predictions)}")
print("="*70)

## Genero Submission

Creo el csv con las predicciones del ensemble.

In [ ]:
# Genero submission.csv
test_filenames = test_dataset.file_paths
ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_filenames]

# Paso probabilidades a clases con threshold 0.5
predictions_binary = (ensemble_predictions > 0.5).astype(int).flatten()

# Creo DataFrame
submission_df = pd.DataFrame({
    'id': ids,
    'label': predictions_binary
})

# Ordeno por id
submission_df = submission_df.sort_values('id')

# Guardo
submission_df.to_csv('submission.csv', index=False)

print("="*70)
print("SUBMISSION.CSV GENERADO")
print("="*70)
print(f"Total predicciones: {len(submission_df)}")
print(f"Cats (0): {(predictions_binary == 0).sum()}")
print(f"Dogs (1): {(predictions_binary == 1).sum()}")
print("="*70)
print("\nPrimeras predicciones:")
print(submission_df.head(10))
print("="*70)
print("\nLISTO para que KAGGLE")
print(f"Técnicas usadas:")
print(f"  - Ensemble: {ENSEMBLE_MODELS}")
print(f"  - TTA: {N_TTA_AUGMENTATIONS} augmentations")
print(f"  - Resolución: {IMG_SIZE}x{IMG_SIZE}")
print(f"  - Total predicciones por imagen: {len(ENSEMBLE_MODELS) * N_TTA_AUGMENTATIONS}")
print("="*70)